# [COPIE POUR S'INSPIRER] Tache 177 - Resultats PFAS UDI ANNUEL

---

#### Objectif : ajout des résultats PFAS par UDI par années

#### Allez vers la fin pour voir le résultat final et les tests


In [14]:
%reload_ext sql
%sql duckdb:///../../database/data.duckdb
%config SqlMagic.displaylimit = 20
%config SqlMagic.named_parameters="enabled" 

Tip: You may define configurations in C:\Users\simon\Documents\Git\13_pollution_eau\pyproject.toml or C:\Users\simon\.jupysql\config.

Did not find user configurations in C:\Users\simon\Documents\Git\13_pollution_eau\pyproject.toml.

In [15]:
%%sql

SELECT *
FROM int__resultats_udi_communes
WHERE categorie = 'pfas'
LIMIT 5

Running query in 'duckdb:///../../database/data.duckdb'

referenceprel,cdparametresiseeaux,limitequal,de_partition,valtraduite,limitequal_float,unite,categorie,cdreseau,inseecommune,datetimeprel
00700187109,PFBA,None,2022,0.000,None,None,pfas,007001817,07039,2022-09-14 09:10:00
00700187109,PFBS,None,2022,0.000,None,None,pfas,007001817,07039,2022-09-14 09:10:00
00700187109,PFDA,None,2022,0.000,None,None,pfas,007001817,07039,2022-09-14 09:10:00
00700187109,PFDODA,None,2022,0.000,None,None,pfas,007001817,07039,2022-09-14 09:10:00
00700187109,PFDODS,None,2022,0.000,None,None,pfas,007001817,07039,2022-09-14 09:10:00


In [22]:
%%sql

SELECT DISTINCT referenceprel, cdparametresiseeaux, cdreseau, inseecommune, COUNT(*) AS nb_analyses
FROM int__resultats_udi_communes
WHERE categorie = 'pfas'
GROUP BY referenceprel, cdparametresiseeaux, cdreseau, inseecommune
HAVING COUNT(*) > 1

Running query in 'duckdb:///../../database/data.duckdb'

referenceprel,cdparametresiseeaux,cdreseau,inseecommune,nb_analyses


In [32]:
%%sql

-- Nombre de résultats PFAS chaque année, doublons compris (UDI et communes)

WITH pfas_prels AS (SELECT de_partition
    FROM
        'int__resultats_udi_communes'
    WHERE
        categorie = 'pfas')

SELECT de_partition, COUNT(*)
FROM pfas_prels
GROUP BY de_partition

Running query in 'duckdb:///../../database/data.duckdb'

de_partition,count_star()
2020,708
2021,214
2022,18638
2023,44543
2024,119555
2025,84843


In [ ]:
%%sql --save pfas_results_udi

WITH pfas_prels AS (
    SELECT *
    FROM
        'int__resultats_udi_communes'
    WHERE
        categorie = 'pfas'),

deduplicated AS (
    SELECT *,
        ROW_NUMBER() OVER (
            PARTITION BY cdreseau, referenceprel, cdparametresiseeaux, datetimeprel
            ORDER BY cdreseau  -- arbitrary choice
        ) AS row_num
    FROM pfas_prels
)

-- Obtention des résultats PFAS par UDI (cdreseau), avec suppression des doublons dus aux communes
SELECT * EXCLUDE (row_num)
FROM deduplicated
WHERE row_num = 1

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

referenceprel,cdparametresiseeaux,limitequal,de_partition,valtraduite,limitequal_float,unite,categorie,cdreseau,inseecommune,datetimeprel
04500164697,PFBS,None,2025,0.000,None,None,pfas,045000553,45331,2025-02-24 10:26:00
04500164697,PFNA,None,2025,0.000,None,None,pfas,045000553,45331,2025-02-24 10:26:00
04500164697,PFTRDS,None,2025,0.000,None,None,pfas,045000553,45331,2025-02-24 10:26:00
04500164283,PFPEA,None,2025,0.000,None,None,pfas,045000562,45344,2025-01-23 09:27:00
04500164283,PFTRDA,None,2025,0.000,None,None,pfas,045000562,45344,2025-01-23 09:27:00
04500164283,SPFAS,"<=0,1 µg/L",2025,0.000,0.10000000149011612,µg/L,pfas,045000562,45344,2025-01-23 09:27:00
04500164200,PFNS,None,2025,0.000,None,None,pfas,045000569,45259,2025-01-20 09:22:00
04500164558,PFDODA,None,2025,0.000,None,None,pfas,045000571,45150,2025-02-11 08:55:00
04500164166,PFHPS,None,2025,0.000,None,None,pfas,045000583,45083,2025-01-17 10:34:00
04500164166,PFPEA,None,2025,0.000,None,None,pfas,045000583,45275,2025-01-17 10:34:00


In [ ]:
%%sql --save pfas_results_udi_vs

WITH pfas_results AS (
    SELECT *
    FROM
        'int__resultats_udi_communes'
    WHERE
        categorie = 'pfas'),

deduplicated_pfas AS (
    SELECT *,
        ROW_NUMBER() OVER (
            PARTITION BY cdreseau, referenceprel, cdparametresiseeaux, datetimeprel
            ORDER BY cdreseau  -- arbitrary choice
        ) AS row_num
    FROM pfas_results
),

valeurs_sanitaires (cdparametresiseeaux, valeur_sanitaire) AS (
    VALUES
        ('PFOA', 0.075),
        ('PFHPA', 0.075),
        ('PFHXA', 960),
        ('PFPEA', 960),
        ('PFBA', 72),
        ('PFBS', 240),
        ('PFOS', 0.18),
        ('PFHXS', 12)
        -- TODO : vérifier unicité de cdparametresiseeaux
)

SELECT
    referenceprel,
    cdreseau,
    dp.cdparametresiseeaux,
    de_partition AS annee,
    datetimeprel,
    valtraduite,
    vs.valeur_sanitaire,
    unite,
    categorie
FROM deduplicated_pfas AS dp
LEFT JOIN valeurs_sanitaires AS vs
    ON dp.cdparametresiseeaux = vs.cdparametresiseeaux
WHERE row_num = 1

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

referenceprel,cdreseau,cdparametresiseeaux,annee,datetimeprel,valtraduite,valeur_sanitaire,unite,categorie
06900167774,069000229,PFBS,2024,2024-10-15 09:24:00,0.000,240.000,None,pfas
06900163179,069000236,PFPEA,2024,2024-02-12 08:41:00,0.002,960.000,None,pfas
06900166081,069000236,PFHXA,2024,2024-08-26 09:10:00,0.000,960.000,None,pfas
06900166081,069000236,PFOS,2024,2024-08-26 09:10:00,0.003,0.180,None,pfas
06900164688,069000272,PFOS,2024,2024-05-27 08:30:00,0.001,0.180,None,pfas
06900166081,069000272,PFPEA,2024,2024-08-26 09:10:00,0.000,960.000,None,pfas
06900167682,069000272,PFHPA,2024,2024-11-18 08:35:00,0.000,0.075,None,pfas
06900164688,069000273,PFBA,2024,2024-05-27 08:30:00,0.000,72.000,None,pfas
06900164688,069000273,PFOS,2024,2024-05-27 08:30:00,0.001,0.180,None,pfas
06900167682,069000273,PFPEA,2024,2024-11-18 08:35:00,0.001,960.000,None,pfas


In [ ]:
%%sql --with pfas_results_udi_vs

SELECT
    referenceprel,
    cdreseau,
    cdparametresiseeaux,
    annee,
    COUNT(*) AS nb_analyses
FROM pfas_results_udi_vs
GROUP BY 1, 2, 3, 4
HAVING COUNT(*) > 1
ORDER BY nb_analyses DESC

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

referenceprel,cdreseau,cdparametresiseeaux,annee,nb_analyses


Plus de doublons dûs aux communes grâce à la window function `ROW_NUMBER()` et au filtre `WHERE row_number = 1`. Cela nous assure d'avoir l'unicité des lignes sur `(referenceprel, cdreseau, cdparametresiseeaux, datetimeprel)`


In [ ]:
%%sql --with pfas_results_udi_vs

SELECT
    referenceprel,
    cdreseau,
    annee,
    COUNT(DISTINCT cdparametresiseeaux) AS nb_parametres,
    -- La somme des 20 PFAS est disponible comme un paramètre (SPFAS)
    MAX(
            CASE WHEN cdparametresiseeaux = 'SPFAS' THEN valtraduite ELSE 0 END
        ) AS sum_20_pfas,
    COUNT(
            DISTINCT CASE
                WHEN cdparametresiseeaux = 'SPFAS' THEN cdparametresiseeaux
            END
        ) AS is_20_pfas,
    SUM(
            CASE
                WHEN
                    cdparametresiseeaux IN ('PFOA', 'PFOS', 'PFNA', 'PFHXS')
                    THEN valtraduite
                ELSE 0
            END
        ) AS sum_4_pfas,
    SUM(
            CASE
                WHEN
                    cdparametresiseeaux IN ('PFOA', 'PFOS', 'PFNA', 'PFHXS')
                    THEN 1
                ELSE 0
            END
        ) AS nb_4_pfas,
    COUNT(
            DISTINCT CASE
                WHEN
                    valeur_sanitaire IS NOT NULL
                    AND valtraduite IS NOT NULL
                    AND valtraduite >= valeur_sanitaire
                    THEN cdparametresiseeaux
            END
        ) AS nb_pfas_above_limit,
    COUNT(
            DISTINCT CASE
                WHEN valtraduite != 0 THEN cdparametresiseeaux
            END
        ) AS nb_quantified_params
FROM pfas_results_udi_vs
GROUP BY referenceprel, cdreseau, annee
HAVING is_20_pfas = 1


Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

referenceprel,cdreseau,annee,nb_parametres,sum_20_pfas,is_20_pfas,sum_4_pfas,nb_4_pfas,nb_pfas_above_limit,nb_quantified_params
00800107746,008000855,2025,21,2.729,1,0.813,4,2,12
00800107747,008000855,2025,21,2.711,1,0.799,4,2,12
00800107542,008000855,2025,21,2.269,1,0.673,4,2,12
01300273206,013001457,2024,21,0.750,1,0.577,4,1,11
01300273540,013001457,2024,21,0.684,1,0.530,4,1,11
01300273882,013001457,2025,21,0.585,1,0.447,4,1,11
01300273898,013001457,2025,21,0.576,1,0.396,4,1,11
01300273793,013001457,2025,21,0.544,1,0.379,4,1,11
06800175863,068006583,2024,21,0.460,1,0.064,4,0,11
06800175159,068006583,2024,21,0.456,1,0.056,4,0,11


Ajout des agrégations avant de réaliser le bilan final.


In [ ]:
%%sql --save pfas_results_udi_agg

WITH pfas_results AS (
    SELECT *,
        ROW_NUMBER() OVER (
            PARTITION BY cdreseau, referenceprel, cdparametresiseeaux, datetimeprel
            ORDER BY cdreseau
        ) AS row_num
    FROM
        'int__resultats_udi_communes'
    WHERE
        categorie = 'pfas'),

valeurs_sanitaires (cdparametresiseeaux, valeur_sanitaire) AS (
    VALUES
        ('PFOA', 0.075),
        ('PFHPA', 0.075),
        ('PFHXA', 960),
        ('PFPEA', 960),
        ('PFBA', 72),
        ('PFBS', 240),
        ('PFOS', 0.18),
        ('PFHXS', 12)
        -- TODO : vérifier unicité de cdparametresiseeaux
),

pfas_results_udi_vs AS (
    SELECT
        referenceprel,
        cdreseau,
        pr.cdparametresiseeaux,
        de_partition AS annee,
        datetimeprel,
        valtraduite,
        vs.valeur_sanitaire,
        unite,
        categorie
FROM pfas_results AS pr
LEFT JOIN valeurs_sanitaires AS vs
    ON pr.cdparametresiseeaux = vs.cdparametresiseeaux
WHERE row_num = 1
)

SELECT
    referenceprel,
    cdreseau,
    annee,
    COUNT(DISTINCT cdparametresiseeaux) AS nb_parametres,
    -- La somme des 20 PFAS est disponible comme un paramètre (SPFAS)
    MAX(
            CASE WHEN cdparametresiseeaux = 'SPFAS' THEN valtraduite ELSE 0 END
        ) AS sum_20_pfas,
    COUNT(
            DISTINCT CASE
                WHEN cdparametresiseeaux = 'SPFAS' THEN cdparametresiseeaux
            END
        ) AS is_20_pfas,
    SUM(
            CASE
                WHEN
                    cdparametresiseeaux IN ('PFOA', 'PFOS', 'PFNA', 'PFHXS')
                    THEN valtraduite
                ELSE 0
            END
        ) AS sum_4_pfas,
    SUM(
            CASE
                WHEN
                    cdparametresiseeaux IN ('PFOA', 'PFOS', 'PFNA', 'PFHXS')
                    THEN 1
                ELSE 0
            END
        ) AS nb_4_pfas,
    COUNT(
            DISTINCT CASE
                WHEN
                    valeur_sanitaire IS NOT NULL
                    AND valtraduite IS NOT NULL
                    AND valtraduite >= valeur_sanitaire
                    THEN cdparametresiseeaux
            END
        ) AS nb_pfas_above_limit,
    COUNT(
            DISTINCT CASE
                WHEN valtraduite != 0 THEN cdparametresiseeaux
            END
        ) AS nb_quantified_params
FROM pfas_results_udi_vs
GROUP BY referenceprel, cdreseau, annee
HAVING is_20_pfas = 1

Your query is using one or more of the following snippets: pfas_results_udi_vs. JupySQL does not support snippet expansion within CTEs yet, CTE generation is disabled

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

referenceprel,cdreseau,annee,nb_parametres,sum_20_pfas,is_20_pfas,sum_4_pfas,nb_4_pfas,nb_pfas_above_limit,nb_quantified_params
00100143331,001000618,2024,21,0.006,1,0.000,4,0,2
00100141737,001000623,2024,21,0.016,1,0.011,4,0,7
00300098553,003000360,2024,21,0.105,1,0.056,4,0,11
00300099031,003000372,2024,21,0.023,1,0.019,4,0,5
00300100487,003000381,2024,21,0.092,1,0.037,4,0,12
00300098549,003001374,2024,21,0.031,1,0.004,4,0,6
00400146637,004000008,2024,21,0.000,1,0.000,4,0,0
00400146884,004000018,2025,21,0.000,1,0.000,4,0,0
00400146886,004000028,2025,21,0.000,1,0.000,4,0,0
00400146875,004000106,2025,21,0.000,1,0.000,4,0,0


In [ ]:
%%sql --with pfas_results_udi_agg

SELECT
    cdreseau,
    annee,
    'pfas' AS categorie,
    'bilan_annuel_' || annee AS periode,
    COUNT(DISTINCT referenceprel) AS nb_prelevements,
    SUM(
        CASE
                WHEN
                        nb_pfas_above_limit > 0
                        THEN 1
                        ELSE 0
        END) AS nb_depassements_vs,
    ROUND((
         SUM(
                CASE
                        WHEN
                                nb_pfas_above_limit > 0
                                THEN 1
                                ELSE 0
                END)
            /
            COUNT(DISTINCT referenceprel)
    ),2) AS ratio_depassements_vs,
    SUM(CASE WHEN sum_20_pfas >= 0.1 THEN 1 ELSE 0 END) AS nb_depassements_20_pfas,
    ROUND((
            SUM(CASE WHEN sum_20_pfas >= 0.1 THEN 1 ELSE 0 END)
            /
            COUNT(DISTINCT referenceprel)
    ),2) AS ratio_depassements_20_pfas,
    SUM(CASE WHEN sum_4_pfas >= 0.1 THEN 1 ELSE 0 END) AS nb_depassements_4_pfas,
    ROUND((
            SUM(CASE WHEN sum_4_pfas >= 0.1 THEN 1 ELSE 0 END)
            /
            COUNT(DISTINCT referenceprel)
    ),2) AS ratio_depassements_4_pfas
FROM pfas_results_udi_agg
GROUP BY cdreseau, annee
ORDER BY nb_prelevements DESC


Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,annee,categorie,periode,nb_prelevements,nb_depassements_vs,ratio_depassements_vs,nb_depassements_20_pfas,ratio_depassements_20_pfas,nb_depassements_4_pfas,ratio_depassements_4_pfas
011000329,2024,pfas,bilan_annuel_2024,53,0,0.0,12,0.23,0,0.0
068001072,2024,pfas,bilan_annuel_2024,25,0,0.0,19,0.76,0,0.0
074000043,2024,pfas,bilan_annuel_2024,20,0,0.0,0,0.0,0,0.0
011004114,2024,pfas,bilan_annuel_2024,19,0,0.0,16,0.84,2,0.11
074001415,2023,pfas,bilan_annuel_2023,17,0,0.0,0,0.0,0,0.0
069000286,2024,pfas,bilan_annuel_2024,14,0,0.0,3,0.21,0,0.0
041000728,2024,pfas,bilan_annuel_2024,13,0,0.0,3,0.23,0,0.0
074001415,2024,pfas,bilan_annuel_2024,12,0,0.0,0,0.0,0,0.0
038000387,2024,pfas,bilan_annuel_2024,12,0,0.0,4,0.33,0,0.0
068006363,2024,pfas,bilan_annuel_2024,12,0,0.0,9,0.75,0,0.0


Ajout de la dernière modification pour obtenir une ligne par couple (cdreseau, annee), qui aggrege le nombre de prélèvements réalisés sur chaque année, le nombre de dépassements (valeurs sanitaires, somme 20 pfas, somme 4 pfas), et le ratio sur le nombre de prélèvements.


---

## RÉSULTAT FINAL


In [ ]:
%%sql --save resultats_pfas_udi_annuel

WITH pfas_results AS (
    SELECT *,
        ROW_NUMBER() OVER (
            PARTITION BY cdreseau, referenceprel, cdparametresiseeaux, datetimeprel
            ORDER BY cdreseau
        ) AS row_num
    FROM
        'int__resultats_udi_communes'
    WHERE
        categorie = 'pfas'),

valeurs_sanitaires (cdparametresiseeaux, valeur_sanitaire) AS (
    VALUES
        ('PFOA', 0.075),
        ('PFHPA', 0.075),
        ('PFHXA', 960),
        ('PFPEA', 960),
        ('PFBA', 72),
        ('PFBS', 240),
        ('PFOS', 0.18),
        ('PFHXS', 12)
        -- TODO : vérifier unicité de cdparametresiseeaux
),

pfas_results_udi_vs AS (
    SELECT
        referenceprel,
        cdreseau,
        pr.cdparametresiseeaux,
        de_partition AS annee,
        datetimeprel,
        valtraduite,
        vs.valeur_sanitaire,
        unite,
        categorie
    FROM pfas_results AS pr
    LEFT JOIN valeurs_sanitaires AS vs
        ON pr.cdparametresiseeaux = vs.cdparametresiseeaux
    WHERE row_num = 1
),

pfas_results_udi_agg AS (
    SELECT
        referenceprel,
        cdreseau,
        annee,
        COUNT(DISTINCT cdparametresiseeaux) AS nb_parametres,
        -- La somme des 20 PFAS est disponible comme un paramètre (SPFAS)
        MAX(
                CASE WHEN cdparametresiseeaux = 'SPFAS' THEN valtraduite ELSE 0 END
            ) AS sum_20_pfas,
        COUNT(
                DISTINCT CASE
                    WHEN cdparametresiseeaux = 'SPFAS' THEN cdparametresiseeaux
                END
            ) AS is_20_pfas,
        SUM(
                CASE
                    WHEN
                        cdparametresiseeaux IN ('PFOA', 'PFOS', 'PFNA', 'PFHXS')
                        THEN valtraduite
                    ELSE 0
                END
            ) AS sum_4_pfas,
        SUM(
                CASE
                    WHEN
                        cdparametresiseeaux IN ('PFOA', 'PFOS', 'PFNA', 'PFHXS')
                        THEN 1
                    ELSE 0
                END
            ) AS nb_4_pfas,
        COUNT(
                DISTINCT CASE
                    WHEN
                        valeur_sanitaire IS NOT NULL
                        AND valtraduite IS NOT NULL
                        AND valtraduite >= valeur_sanitaire
                        THEN cdparametresiseeaux
                END
            ) AS nb_pfas_above_limit,
        COUNT(
                DISTINCT CASE
                    WHEN valtraduite != 0 THEN cdparametresiseeaux
                END
            ) AS nb_quantified_params
    FROM pfas_results_udi_vs
    GROUP BY referenceprel, cdreseau, annee
    HAVING is_20_pfas = 1
)

SELECT
    cdreseau,
    annee,
    'pfas' AS categorie,
    'bilan_annuel_' || annee AS periode,
    COUNT(DISTINCT referenceprel) AS nb_prelevements,
    SUM(
        CASE
                WHEN
                        nb_pfas_above_limit > 0
                        THEN 1
                        ELSE 0
        END) AS nb_depassements_vs,
    ROUND((
         SUM(
                CASE
                        WHEN
                                nb_pfas_above_limit > 0
                                THEN 1
                                ELSE 0
                END)
            /
            COUNT(DISTINCT referenceprel)
    ),2) AS ratio_depassements_vs,
    SUM(CASE WHEN sum_20_pfas >= 0.1 THEN 1 ELSE 0 END) AS nb_depassements_20_pfas,
    ROUND((
            SUM(CASE WHEN sum_20_pfas >= 0.1 THEN 1 ELSE 0 END)
            /
            COUNT(DISTINCT referenceprel)
    ),2) AS ratio_depassements_20_pfas,
    SUM(CASE WHEN sum_4_pfas >= 0.1 THEN 1 ELSE 0 END) AS nb_depassements_4_pfas,
    ROUND((
            SUM(CASE WHEN sum_4_pfas >= 0.1 THEN 1 ELSE 0 END)
            /
            COUNT(DISTINCT referenceprel)
    ),2) AS ratio_depassements_4_pfas
FROM pfas_results_udi_agg
GROUP BY cdreseau, annee
ORDER BY nb_prelevements DESC

Your query is using one or more of the following snippets: pfas_results_udi_agg, pfas_results_udi_vs. JupySQL does not support snippet expansion within CTEs yet, CTE generation is disabled

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,annee,categorie,periode,nb_prelevements,nb_depassements_vs,ratio_depassements_vs,nb_depassements_20_pfas,ratio_depassements_20_pfas,nb_depassements_4_pfas,ratio_depassements_4_pfas
011000329,2024,pfas,bilan_annuel_2024,53,0,0.0,12,0.23,0,0.0
068001072,2024,pfas,bilan_annuel_2024,25,0,0.0,19,0.76,0,0.0
074000043,2024,pfas,bilan_annuel_2024,20,0,0.0,0,0.0,0,0.0
011004114,2024,pfas,bilan_annuel_2024,19,0,0.0,16,0.84,2,0.11
074001415,2023,pfas,bilan_annuel_2023,17,0,0.0,0,0.0,0,0.0
069000286,2024,pfas,bilan_annuel_2024,14,0,0.0,3,0.21,0,0.0
041000728,2024,pfas,bilan_annuel_2024,13,0,0.0,3,0.23,0,0.0
068006363,2024,pfas,bilan_annuel_2024,12,0,0.0,9,0.75,0,0.0
069000302,2024,pfas,bilan_annuel_2024,12,0,0.0,2,0.17,0,0.0
069000295,2024,pfas,bilan_annuel_2024,12,0,0.0,9,0.75,0,0.0


---

## TESTS SUR LES RÉSULTATS OBTENUS


In [ ]:
%%sql --with resultats_pfas_udi_annuel

SELECT
    annee,
    ROUND(AVG(nb_prelevements),2) AS avg_nb_prel,
    ROUND(AVG(nb_depassements_vs),2) AS avg_nb_dep_vs,
    ROUND(AVG(nb_depassements_20_pfas),2) AS avg_nb_dep_20_pfas,
    ROUND(AVG(nb_depassements_4_pfas),2) AS avg_nb_dep_4_pfas
FROM resultats_pfas_udi_annuel
GROUP BY annee

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

annee,avg_nb_prel,avg_nb_dep_vs,avg_nb_dep_20_pfas,avg_nb_dep_4_pfas
2022,2.75,0.03,0.66,0.03
2023,3.8,0.0,0.47,0.02
2024,2.39,0.0,0.21,0.02
2025,1.19,0.01,0.03,0.01


---

#### Réalisation de tests sur cdreseau `011000329` (le plus testé concernant les PFAS)


In [ ]:
%%sql

SELECT COUNT(DISTINCT referenceprel) AS nb_prelevements
FROM 'int__resultats_udi_communes'
WHERE
    cdreseau = '011000329'
    AND de_partition = 2024
    AND categorie = 'pfas'

Running query in 'duckdb:///../../database/data.duckdb'

nb_prelevements
53


In [ ]:
%%sql

SELECT *
FROM 'int__resultats_udi_communes'
WHERE
    cdreseau = '011000329'
    AND de_partition = 2024
    AND cdparametresiseeaux = 'SPFAS'
ORDER BY valtraduite DESC

Running query in 'duckdb:///../../database/data.duckdb'

referenceprel,cdparametresiseeaux,limitequal,de_partition,valtraduite,limitequal_float,unite,categorie,cdreseau,inseecommune,datetimeprel
01100170013,SPFAS,"<=0,1 µg/L",2024,0.151,0.10000000149011612,µg/L,pfas,011000329,11262,2024-02-02 09:21:00
01100170030,SPFAS,"<=0,1 µg/L",2024,0.139,0.10000000149011612,µg/L,pfas,011000329,11262,2024-02-26 13:06:00
01100170031,SPFAS,"<=0,1 µg/L",2024,0.134,0.10000000149011612,µg/L,pfas,011000329,11262,2024-02-26 12:37:00
01100169656,SPFAS,"<=0,1 µg/L",2024,0.131,0.10000000149011612,µg/L,pfas,011000329,11262,2024-12-19 12:06:00
01100167266,SPFAS,"<=0,1 µg/L",2024,0.129,0.10000000149011612,µg/L,pfas,011000329,11262,2024-07-24 09:39:00
01100167934,SPFAS,"<=0,1 µg/L",2024,0.128,0.10000000149011612,µg/L,pfas,011000329,11262,2024-09-24 09:42:00
01100169360,SPFAS,"<=0,1 µg/L",2024,0.126,0.10000000149011612,µg/L,pfas,011000329,11262,2024-10-21 11:46:00
01100170034,SPFAS,"<=0,1 µg/L",2024,0.120,0.10000000149011612,µg/L,pfas,011000329,11262,2024-02-26 11:40:00
01100170033,SPFAS,"<=0,1 µg/L",2024,0.107,0.10000000149011612,µg/L,pfas,011000329,11262,2024-02-26 12:20:00
01100167684,SPFAS,"<=0,1 µg/L",2024,0.106,0.10000000149011612,µg/L,pfas,011000329,11262,2024-08-26 10:50:00


On a bien 12 valeurs SPFAS ≥ 0.1 !


---
# METAUX LOURDS

Seuls l'Arsenic (As) et le Plomb (Pb) sont pris en compte dans la catégorie des métaux lourds.  
La catégorie "métaux lourds" n'est donc pas utilisée et on pourra choisir sur le site As ou Pb.  
Les requêtes sont donc créées en ce sens : une pour chaque substance.

In [1]:
%load_ext sql
%sql duckdb:///../../database/data.duckdb
%config SqlMagic.displaylimit = 100

Tip: You may define configurations in C:\Users\simon\Documents\Git\13_pollution_eau\pyproject.toml or C:\Users\simon\.jupysql\config.

Did not find user configurations in C:\Users\simon\Documents\Git\13_pollution_eau\pyproject.toml.

Connecting to 'duckdb:///../../database/data.duckdb'

# Test de requête autres polluants

In [2]:
%%sql
select * from int__resultats_udi_communes where inseecommune = '07194' limit 5

Running query in 'duckdb:///../../database/data.duckdb'

referenceprel,cdparametresiseeaux,limitequal,de_partition,valtraduite,limitequal_float,unite,categorie,cdreseau,inseecommune,datetimeprel
00700169098,1112TCE,None,2020,0.000,None,None,hydrocarbures,007001388,07194,2020-06-17 10:36:00
00700169098,111TCL,None,2020,0.000,None,None,hydrocarbures,007001388,07194,2020-06-17 10:36:00
00700169098,1122TCE,None,2020,0.000,None,None,hydrocarbures,007001388,07194,2020-06-17 10:36:00
00700169098,112TCE,None,2020,0.000,None,None,hydrocarbures,007001388,07194,2020-06-17 10:36:00
00700169098,11DCET,None,2020,0.000,None,None,hydrocarbures,007001388,07194,2020-06-17 10:36:00


## Requête nitrates (resultats_nitrates.ipynb) pour s'en inspirer

In [3]:
%%sql --save int__resultats_nitrites_udi_dernier
WITH
last_pvl AS (
    SELECT
        cdreseau,
        categorie,
        cdparametresiseeaux,
        datetimeprel,
        valtraduite,
        ROW_NUMBER()
            OVER (
                PARTITION BY cdreseau, cdparametresiseeaux
                ORDER BY datetimeprel DESC
            )
            AS row_number
    FROM
        int__resultats_udi_communes
    WHERE
        categorie = 'nitrite'
        AND
        -- On garde les prélèvements de moins d'un an
        CURRENT_DATE - datetimeprel < INTERVAL 1 YEAR
),

split_nitrites AS (
    SELECT
        last_pvl.cdreseau,
        last_pvl.categorie,
        COUNT(DISTINCT last_pvl.cdparametresiseeaux) AS nb_parametres,
        MAX(last_pvl.datetimeprel) AS dernier_prel_datetime,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3'
                    THEN last_pvl.datetimeprel
            END
        ) AS dernier_prel_datetime_n03,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO2'
                    THEN last_pvl.datetimeprel
            END
        ) AS dernier_prel_datetime_n02,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3_NO2'
                    THEN last_pvl.datetimeprel
            END
        ) AS dernier_prel_datetime_no3_no2,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3'
                    THEN last_pvl.valtraduite
            END
        ) AS valtraduite_no3,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO2'
                    THEN last_pvl.valtraduite
            END
        ) AS valtraduite_no2,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3_NO2'
                    THEN last_pvl.valtraduite
            END
        ) AS valtraduite_no3_no2
    FROM
        last_pvl
    WHERE
        last_pvl.row_number = 1
    GROUP BY
        last_pvl.cdreseau,
        last_pvl.categorie
)

SELECT
    split_nitrites.cdreseau,
    split_nitrites.categorie,
    'dernier_prel' AS periode,
    split_nitrites.dernier_prel_datetime,
    split_nitrites.nb_parametres,
    CASE
        WHEN
            DATE_DIFF(
                'day',
                LEAST(
                    split_nitrites.dernier_prel_datetime_n03,
                    split_nitrites.dernier_prel_datetime_n02,
                    split_nitrites.dernier_prel_datetime_no3_no2
                ),
                GREATEST(
                    split_nitrites.dernier_prel_datetime_n03,
                    split_nitrites.dernier_prel_datetime_n02,
                    split_nitrites.dernier_prel_datetime_no3_no2
                )
            ) >= 30
            OR split_nitrites.valtraduite_no2 IS NULL
            OR split_nitrites.valtraduite_no3 IS NULL
            THEN 'donnee_manquante'
        WHEN
            split_nitrites.valtraduite_no3_no2 IS NULL
            AND split_nitrites.valtraduite_no2 IS NOT NULL
            AND split_nitrites.valtraduite_no3 IS NOT NULL
            AND split_nitrites.valtraduite_no3 < 50
            AND split_nitrites.valtraduite_no2 < 0.5
            AND split_nitrites.valtraduite_no3 / 50
            + split_nitrites.valtraduite_no2 / 3
            < 1
            THEN 'conforme'
        WHEN
            split_nitrites.nb_parametres = 3
            AND split_nitrites.valtraduite_no3 < 50
            AND split_nitrites.valtraduite_no2 < 0.5
            AND split_nitrites.valtraduite_no3_no2 < 1
            THEN 'conforme'
        WHEN
            split_nitrites.valtraduite_no3 >= 50
            OR split_nitrites.valtraduite_no2 >= 0.5
            OR split_nitrites.valtraduite_no3_no2 >= 1
            THEN 'non_conforme'
        /*        WHEN
            split_nitrites.nb_parametres != 3
            AND (
                split_nitrites.valtraduite_no3 < 50
                OR split_nitrites.valtraduite_no2 < 0.5
                OR split_nitrites.valtraduite_no3_no2 < 1
            )
            THEN 'non_quantifie'*/
        ELSE 'error'
    END AS resultat
FROM
    split_nitrites

Running query in 'duckdb:///../../database/data.duckdb'

cdreseau,categorie,periode,dernier_prel_datetime,nb_parametres,resultat
001000294,nitrite,dernier_prel,2024-07-17 11:14:00,3,conforme
001000407,nitrite,dernier_prel,2024-09-26 08:38:00,3,conforme
001000860,nitrite,dernier_prel,2024-11-22 11:19:00,3,conforme
002000067,nitrite,dernier_prel,2024-12-06 08:12:00,2,donnee_manquante
002000091,nitrite,dernier_prel,2025-01-17 09:22:00,3,conforme
002000693,nitrite,dernier_prel,2025-01-06 08:59:00,3,conforme
003000355,nitrite,dernier_prel,2024-11-25 08:35:00,3,conforme
004000089,nitrite,dernier_prel,2025-01-30 08:36:00,3,donnee_manquante
004000142,nitrite,dernier_prel,2024-09-03 11:36:00,3,conforme
004000332,nitrite,dernier_prel,2024-09-09 09:03:00,2,conforme


# Requête métaux lourds : Plomb et Arsenic

## Obtention du libellé exact pour les métaux lourds

In [4]:
%%sql
SELECT DISTINCT
    categorie,
FROM
    int__resultats_udi_communes
;

Running query in 'duckdb:///../../database/data.duckdb'

categorie
pesticides
métaux lourds
cvm
perchlorate
hydrocarbures
nitrite
pfas


## Sélection des résultats de prélèvements pour tous les métaux lourds

In [5]:
%%sql

SELECT
    *
FROM
    int__resultats_udi_communes
WHERE
    categorie = 'métaux lourds'
LIMIT 5
;

Running query in 'duckdb:///../../database/data.duckdb'

referenceprel,cdparametresiseeaux,limitequal,de_partition,valtraduite,limitequal_float,unite,categorie,cdreseau,inseecommune,datetimeprel
00200179914,AS,<=10 µg/L,2020,0.210,10.0,µg/L,métaux lourds,002001716,02691,2020-05-05 15:27:00
00200179914,ASD,<=10 µg/L,2020,0.210,10.0,µg/L,métaux lourds,002001716,02691,2020-05-05 15:27:00
00200179914,CD,<=5 µg/L,2020,0.000,5.0,µg/L,métaux lourds,002001716,02691,2020-05-05 15:27:00
00200179914,CDD,<=5 µg/L,2020,0.000,5.0,µg/L,métaux lourds,002001716,02691,2020-05-05 15:27:00
00200179914,CRD,<=50.0 µg/L,2020,0.800,50.0,µg/L,métaux lourds,002001716,02691,2020-05-05 15:27:00


## Récupération des cdparametresiseeaux pour identifier Arsenic et Plomb

In [6]:
%%sql

SELECT DISTINCT
    cdparametresiseeaux
FROM
    int__resultats_udi_communes
WHERE
    categorie = 'métaux lourds'
;

Running query in 'duckdb:///../../database/data.duckdb'

cdparametresiseeaux
SB
NI
ASD
CDD
PB210
BRANCPB
CR6
CRD
SN
SE


--> AS et PB

## Requête Plomb

In [33]:
%%sql

SELECT
    *
FROM
    int__resultats_udi_communes
WHERE
    categorie = 'métaux lourds'
    AND cdparametresiseeaux = 'PB'
LIMIT 5
;

Running query in 'duckdb:///../../database/data.duckdb'

referenceprel,cdparametresiseeaux,limitequal,de_partition,valtraduite,limitequal_float,unite,categorie,cdreseau,inseecommune,datetimeprel
01000096568,PB,<=10 µg/L,2022,5.600,10.0,µg/L,métaux lourds,010003088,10416,2022-10-24 12:13:00
01100156670,PB,<=10 µg/L,2022,0.000,10.0,µg/L,métaux lourds,011000418,11244,2022-11-23 09:35:00
01100154988,PB,<=10 µg/L,2022,8.000,10.0,µg/L,métaux lourds,011000537,11339,2022-06-20 10:59:00
01200107199,PB,<=10 µg/L,2022,0.170,10.0,µg/L,métaux lourds,012000747,12068,2022-07-25 14:30:00
01200107199,PB,<=10 µg/L,2022,0.170,10.0,µg/L,métaux lourds,012000747,12102,2022-07-25 14:30:00


### Vérification unicité prélèvement (referenceprel)

In [34]:
%%sql

SELECT DISTINCT de_partition as annee, referenceprel, cdparametresiseeaux, cdreseau, inseecommune, COUNT(*) AS nb_analyses
FROM int__resultats_udi_communes
WHERE categorie = 'métaux lourds' AND cdparametresiseeaux = 'PB'
GROUP BY annee, referenceprel, cdparametresiseeaux, cdreseau, inseecommune
HAVING COUNT(*) > 1
LIMIT 10

Running query in 'duckdb:///../../database/data.duckdb'

annee,referenceprel,cdparametresiseeaux,cdreseau,inseecommune,nb_analyses
2021,04400226978,PB,044000138,44109,2
2024,07200139547,PB,072000066,72381,2
2021,04400226978,PB,044000138,44026,2
2024,07200139547,PB,072003592,72008,2
2021,04400226978,PB,044000138,44204,2
2021,97600027789,PB,976000061,97602,2
2021,04400226978,PB,044000169,44184,2
2021,04400226978,PB,044000169,44176,2
2024,07200139547,PB,072003592,72065,2
2024,07200139547,PB,072003592,72095,2


--> Pas d'unicité sur certaines referenceprel pour PB.  
**Attention pour la création de la requête**

#### Exemple de doublons

In [38]:
%%sql

SELECT
    *
FROM
    int__resultats_udi_communes
WHERE
    de_partition = 2021
    AND categorie = 'métaux lourds'
    AND cdparametresiseeaux = 'PB'
    AND referenceprel = '04400226978'
    AND cdreseau = '044000169'
    AND inseecommune = '44151'
;

Running query in 'duckdb:///../../database/data.duckdb'

referenceprel,cdparametresiseeaux,limitequal,de_partition,valtraduite,limitequal_float,unite,categorie,cdreseau,inseecommune,datetimeprel
04400226978,PB,<=10 µg/L,2021,0.000,10.0,µg/L,métaux lourds,044000169,44151,2021-11-16 11:40:00
04400226978,PB,<=10 µg/L,2021,0.000,10.0,µg/L,métaux lourds,044000169,44151,2021-11-16 11:40:00


### Nombre de résultats Plomb chaque année, doublons compris (UDI et communes)

In [40]:
%%sql

WITH pb_prels AS (
    SELECT
        de_partition
    FROM
        'int__resultats_udi_communes'
    WHERE
        cdparametresiseeaux = 'PB'
    )

SELECT de_partition, COUNT(*)
FROM pb_prels
GROUP BY de_partition
;

Running query in 'duckdb:///../../database/data.duckdb'

de_partition,count_star()
2020,52653
2021,50380
2022,48543
2023,52014
2024,53140
2025,8332


In [44]:
%%sql --save pfas_results_udi

WITH pb_prels AS (
    SELECT 
        *
    FROM
        'int__resultats_udi_communes'
    WHERE
        cdparametresiseeaux = 'PB'
),

deduplicated AS (
    SELECT 
        *,
        ROW_NUMBER() OVER (
            PARTITION BY cdreseau, referenceprel, cdparametresiseeaux, datetimeprel
            ORDER BY cdreseau  -- arbitrary choice
        ) AS row_num
    FROM pb_prels
)

-- Obtention des résultats PB par UDI (cdreseau), avec suppression des doublons dus aux communes
SELECT * EXCLUDE (row_num)
FROM deduplicated
WHERE row_num = 1
;

Running query in 'duckdb:///../../database/data.duckdb'

referenceprel,cdparametresiseeaux,limitequal,de_partition,valtraduite,limitequal_float,unite,categorie,cdreseau,inseecommune,datetimeprel
03700128520,PB,<=10 µg/L,2024,4.200,10.0,µg/L,métaux lourds,037000722,37281,2024-09-02 09:57:00
03700129549,PB,<=10 µg/L,2024,7.900,10.0,µg/L,métaux lourds,037000723,37270,2024-12-05 11:10:00
03700124002,PB,<=10 µg/L,2023,0.700,10.0,µg/L,métaux lourds,037000741,37282,2023-08-09 10:19:00
03700125006,PB,<=10 µg/L,2023,0.700,10.0,µg/L,métaux lourds,037000746,37233,2023-10-17 09:44:00
03700114780,PB,<=10 µg/L,2021,0.830,10.0,µg/L,métaux lourds,037000747,37233,2021-05-05 10:40:00
03700129935,PB,<=10 µg/L,2025,0.000,10.0,µg/L,métaux lourds,037000747,37233,2025-01-14 09:40:00
03700111289,PB,<=10 µg/L,2020,0.660,10.0,µg/L,métaux lourds,037000774,37208,2020-06-03 09:00:00
03700125657,PB,<=10 µg/L,2023,7.300,10.0,µg/L,métaux lourds,037000776,37124,2023-11-28 10:44:00
03700127129,PB,<=10 µg/L,2024,0.500,10.0,µg/L,métaux lourds,037000782,37156,2024-05-23 10:23:00
03700126104,PB,<=10 µg/L,2024,0.800,10.0,µg/L,métaux lourds,037000791,37163,2024-02-06 11:25:00
